#   TP1  - Horário Semanal 
## Outubro, 2022

Bruno Miguel Ferreira Fernandes - a95972

Hugo Filipe de Sá Rocha - a96463

## Variáveis
#### Inputs do Problema:

- $P$ - Projetos

- $C$ - Colaboradores

- $D$ - Dias

- $T$ - Slots

- $S$ - Salas

### Auxiliares

- $ alocP_{p, s, d, t}$ - representa a existência de uma reunião de um projeto p, numa determinada sala s,num determinado dia d e num determinado slot t;

- $ alocC_{c,p,d,t} $ - representa a alocação de um colaborador c a uma reunião de um projeto p, num determinado dia d e slot t;

- $ disp_{c,(d,t)} $ - representa a disponibilidade do colaborador c no dia d, no slot t;

- $rsm_{c}$ - representa o número de reuniões semanais do colaborador c;


### Condições
Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder.

1. O líder do projeto participa em todas as reuniões do seu projeto.

2. Os restantes colaboradores podem ou não participar consoante disponibilidade, num mínimo (“quorum”) de  $50\%$ do total de colaboradores do projeto.

3. Cada projeto tem um determinado número de reuniões semanais.

4. Um colaborador só participa nos projetos em que está incluído.

5. Um colaborador só está presente numa reunião de cada vez.

6. A mesma sala só pode ser usada em uma reunião em cada slot.

7. Número de reuniões totais do colaborador c. 

### Critérios de optimização

1.Maximizar o número de reuniões efetivamente realizadas.

2.Minimizar o número médio de reuniões por participante.

## Definir valores para os inputs do Problema

Começamos por importar a biblioteca de programação linear do OR-Tools e criar uma instância do solver.

Depois inicializámos o solver e definimos os valores para as constantes.

In [49]:
from ortools.linear_solver import pywraplp
import random

#iniciar solver
solver = pywraplp.Solver.CreateSolver('SCIP')


P = 4; #projetos
C = 8; #colaboradores
D = 5; #dias
T = 6; #slots
S = 4; #salas

# projeto -> [reuniões,líder,[colaboradores]]
projetos = {
    1: [20, 6, [6,4,7,3,8,5]],
    2: [15, 3, [3,5,4,7]],
    3: [9, 1, [1,2,4]],
    4: [3, 8, [8,3]]
}

"""
P = 8; 
C = 9;
D = 5; 
T = 6;
S = 4; 


projetos = {
    1: [3, 1, [1,5]],
    2: [2, 7, [7,4]],
    3: [6, 3, [3,2,6,8]],
    4: [3, 8, [8,1]],
    5: [1, 9, [9]],
    6: [3, 4, [4,5,6]],
    7: [4, 2, [2,9,7,3]],
    8: [9, 5, [5,1,2,4,8]] }


P = 3; 
C = 9;
D = 5; 
T = 6;
S = 3; 


projetos = {
    1: [30, 4, [4,1,3,9,8,5]],
    2: [20, 1, [1,2,6,7,4]],
    3: [25, 9, [9,8,3,6,2,7]] }

"""


'\nP = 8; \nC = 9;\nD = 5; \nT = 6;\nS = 4; \n\n\nprojetos = {\n    1: [3, 1, [1,5]],\n    2: [2, 7, [7,4]],\n    3: [6, 3, [3,2,6,8]],\n    4: [3, 8, [8,1]],\n    5: [1, 9, [9]],\n    6: [3, 4, [4,5,6]],\n    7: [4, 2, [2,9,7,3]],\n    8: [9, 5, [5,1,2,4,8]] }\n\n\nP = 3; \nC = 9;\nD = 5; \nT = 6;\nS = 3; \n\n\nprojetos = {\n    1: [30, 4, [4,1,3,9,8,5]],\n    2: [20, 1, [1,2,6,7,4]],\n    3: [25, 9, [9,8,3,6,2,7]] }\n\n'

In [50]:
#matriz booleana disponibilidade (proporção 80:20)
#80% de chance de estar disponível e 20% de não estar


disp = {}
for c in range(1, C+1):
    disp[c] = {}
    for d in range(1,D+1):
        for t in range(1,T+1):
            x = random.randint(0,100)
            if x>=20:
                disp[c][(d,t)] = 1
            else:
                disp[c][(d,t)] = 0

# alocP[p][s][d][t] == 1 sse existe reunião do projeto p na sala s no dia d no slot t           
alocP = {}         
for p in range(1,P+1):
         for s in range(1, S+1):
            for d in range(1,D+1):
                    for t in range(1,T+1):
                         alocP[p,s,d,t] = solver.BoolVar(f'alocP[{p},{s},{d},{t}]')

# alocC[c][p][d][t] == 1 sse colaborador c esta presente na reuniao do projeto p 
                                #do dia d do slot t
alocC = {}
for c in range(1, C+1):
    for p in range(1,P+1):
        for d in range(1,D+1):
            for t in range(1,T+1):
                alocC[c,p,d,t] = solver.BoolVar(f'alocC[{c},{p},{d},{t}]')

# Reunião por participante (colaborador)
rsm = {}
for c in range(1,C+1):
    rsm[c] = solver.IntVar(-solver.infinity(), solver.infinity(), f'rsm[{c}]')


## Modelação das restrições e sua introdução no Solver

1. Líder tem de participar em todas as reuniões do projeto.

$ \forall_{1\leq p \leq P} \centerdot \forall_{1\leq d \leq D} \centerdot \forall_{1\leq t \leq T}  (\displaystyle\sum_{1 \leq s \leq S} alocP_{p,s,d,t} ) == alocC_{lider,p,d,t} $

In [51]:

for p in range(1,P+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum([alocP[p,s,d,t] for s in range(1,S+1)]) == 
                       alocC[projetos[p][1],p,d,t])
      

2. Mínimo de 50% dos colaboradores têm de participar nas reuniões do projeto.

$ \forall_{1\leq p \leq P} \centerdot \forall_{1\leq s \leq S}\centerdot \forall_{1\leq d \leq D} \centerdot \forall_{1\leq t \leq T}\centerdot (\displaystyle\sum_{1 \leq c \leq C} alocC_{c,p,d,t} ) \leq (0.5*len(colaboradores)*alocP_{p,s,d,t}) $

In [52]:
for p in range(1,P+1):
    for s in range(1,S+1):
        for d in range(1,D+1):
            for t in range(1,T+1):
                solver.Add(sum([alocC[c,p,d,t] for c in projetos[p][2]]) >= 
                               0.5*len(projetos[p][2])*alocP[p,s,d,t])

3. Cada projeto tem um determinado número de reuniões semanais.



$ \forall_{1\leq p \leq P} \centerdot (\displaystyle\sum_{1 \leq s \leq S,\\ 1 \leq d \leq D,\\ 1 \leq t \leq T } alocP_{p,s,d,t} ) \geq projetos[p][0] $

In [53]:
for p in range(1,P+1):
    solver.Add(sum([alocP[p,s,d,t] for s in range(1, S+1) for d in range(1, D+1) 
                       for t in range(1, T+1)]) >= projetos[p][0])


4. Um colaborador só participa nos projetos em que está incluído. 



$ \forall_{1\leq c \leq C} \centerdot \forall_{1\leq d \leq D}\centerdot \forall_{1\leq t \leq T} \centerdot \forall_{1\leq p \leq P} \centerdot if c \notin projetos[p][2]: \ alocC_{c,p,d,t} == 0$

In [54]:
for c in range(1, C+1):
    for d in range(1, D+1):
        for t in range(1, T+1):
            for p in range(1,P+1):
                if c not in projetos[p][2]:
                    solver.Add(alocC[c,p,d,t] == 0)

5. Um colaborador só está presente numa reunião de cada vez.

$ \forall_{1 \leq c \leq C} \centerdot \forall_{1\leq d \leq D}\centerdot \forall_{1\leq t \leq T} (\displaystyle\sum_{1 \leq p \leq P} alocC_{c,p,d,t}) \leq 1$

In [55]:
for c in range(1, C+1):
    for d in range(1, D+1):
        for t in range(1,T+1):
            solver.Add(sum([alocC[c,p,d,t] for p in range(1,P+1)]) <= 1)

6. A mesma sala só pode ser usada em uma reunião em cada slot.

$ \forall_{1\leq s \leq S} \centerdot \forall_{1\leq d \leq D}\centerdot \forall_{1\leq t \leq T} (\displaystyle\sum_{1 \leq p \leq P} alocP_{p,s,d,t} ) \leq 1 $

In [56]:
for s in range(1,S+1):
    for d in range(1,D+1):
        for t in range(1,T+1):
            solver.Add(sum([alocP[p,s,d,t] for p in range(1,P+1)]) <= 1)

7. Número de reuniões totais do colaborador c.



$ \forall_{1 \leq c \leq C} \centerdot  rsm_{c} == (\displaystyle\sum_{1 \leq p \leq P, 1 \leq s \leq S,\\ 1 \leq d \leq D, 1 \leq t \leq T } alocC_{c,p,d,t}) $

In [57]:
for c in range(1,C+1):
    solver.Add(rsm[c] == sum([alocC[c,p,d,t] for p in range(1,P+1) 
            for s in range (1,S+1) for d in range(1,D+1) for t in range (1,T+1)]))
                                

## Optimização do problema

1. Maximizar o número de reuniões realizadas.

In [58]:
solver.Maximize(sum([ alocP[p,s,d,t] for p in range(1,P+1) for s in range (1,S+1) 
                       for d in range(1,D+1) for t in range (1,T+1)]))

2. Minimizar número médio de reuniões por participante.

In [59]:
solver.Minimize(sum([rsm[c] for c in range(1,C+1)])/C)

Imprimindo os respetivos projetos e salas nos dias e slots respetivos bem como os colaboradores envolvidos.

In [60]:
if solver.Solve() == pywraplp.Solver.OPTIMAL:
    print("Solução encontrada\n\n")
    for p in range(1,P+1):
         for s in range(1, S+1):
            for d in range(1,D+1):
                for t in range(1,T+1):
                    if(alocP[p,s,d,t].solution_value() == 1):
                        print(f'Reunião do projeto {p} na sala {s} no dia {d} no slot {t}')
                        print("Colaboradores: ")
                        for c in range(1,C+1):
                            if alocC[c,p,d,t].solution_value() == 1:
                                print(f'{c}')
                        print("\n")
                        
else:
    print("Não foi possível resolver o problema\n")

Solução encontrada


Reunião do projeto 1 na sala 2 no dia 3 no slot 6
Colaboradores: 
6
7
8


Reunião do projeto 1 na sala 2 no dia 4 no slot 6
Colaboradores: 
3
5
6


Reunião do projeto 1 na sala 2 no dia 5 no slot 2
Colaboradores: 
5
6
7


Reunião do projeto 1 na sala 3 no dia 1 no slot 1
Colaboradores: 
3
6
8


Reunião do projeto 1 na sala 3 no dia 2 no slot 2
Colaboradores: 
3
6
8


Reunião do projeto 1 na sala 3 no dia 2 no slot 6
Colaboradores: 
6
7
8


Reunião do projeto 1 na sala 3 no dia 5 no slot 1
Colaboradores: 
5
6
7


Reunião do projeto 1 na sala 3 no dia 5 no slot 3
Colaboradores: 
5
6
8


Reunião do projeto 1 na sala 3 no dia 5 no slot 5
Colaboradores: 
5
6
8


Reunião do projeto 1 na sala 4 no dia 1 no slot 2
Colaboradores: 
6
7
8


Reunião do projeto 1 na sala 4 no dia 1 no slot 3
Colaboradores: 
3
6
7


Reunião do projeto 1 na sala 4 no dia 1 no slot 4
Colaboradores: 
4
6
8


Reunião do projeto 1 na sala 4 no dia 1 no slot 5
Colaboradores: 
5
6
7


Reunião do projet